In [460]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [461]:
train_df=pd.read_excel("/home/hackathon/data/hackathon_train_set_tabular_2.xlsx")

In [462]:
test_df=train_df.drop(['ner'], axis=1)
test_id = test_df['unique_tok_id']

In [463]:
nere=[]



for i,row in train_df.iterrows():
    if row["ner"]=="O":
        nere.append(0)
    else:
        nere.append(1)
train_df["nere"]=nere
lol=train_df.groupby("sentence_num")["nere"].sum()
train_df["sum"]=train_df["sentence_num"].map(lol)



In [464]:
#train_df

In [465]:
#test_df=train_df[train_df["sum"]==2]

In [466]:
#train_df=train_df[train_df['sum']!=2]
# train_df.drop("sum",axis=1,inplace=True)
# test_df.drop("sum",axis=1,inplace=True)

In [467]:
#justfortest=test_df["ner"]

In [468]:
justtrain=train_df[train_df["sum"]==0]

In [469]:
justtrain.shape

(12053, 7)

In [470]:
justtrain1=justtrain.iloc[0:5500,:]

In [471]:
train_df=train_df[train_df['sum']!=0]


In [472]:
train_df.head()

,doc_num,sentence_num,unique_tok_id,word,ner,nere,sum
17,1,6,18,Keandris,B-NameDefaulter,1,2
18,1,6,19,Hogan,L-NameDefaulter,1,2
19,1,7,20,Located,O,0,4
20,1,7,21,at,O,0,4
21,1,7,22,:,O,0,4


In [473]:
train_df=pd.concat([train_df,justtrain1],axis=0)

In [474]:
train_df.head()

,doc_num,sentence_num,unique_tok_id,word,ner,nere,sum
17,1,6,18,Keandris,B-NameDefaulter,1,2
18,1,6,19,Hogan,L-NameDefaulter,1,2
19,1,7,20,Located,O,0,4
20,1,7,21,at,O,0,4
21,1,7,22,:,O,0,4


In [475]:
train_df.drop(["nere","sum"],axis=1,inplace=True)
#test_df.drop(["nere","sum","ner"],axis=1,inplace=True)

In [476]:
train_df['word'][40]

'pertaining'

In [391]:
test_df.head()

,doc_num,sentence_num,unique_tok_id,word
17,1,6,18,Keandris
18,1,6,19,Hogan
28,1,8,29,Georgia
29,1,8,30,30032
261,1,27,262,Served


In [392]:
X = train_df.drop(['unique_tok_id','ner','doc_num'], axis=1)
v = DictVectorizer(sparse=True)
X = v.fit_transform(X.to_dict('records'))

In [393]:
y = train_df.ner.values

In [394]:
classes = np.unique(y)
classes = classes.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)
X_train.shape, y_train.shape

((9993, 2562), (9993,))

In [395]:
nb = MultinomialNB(alpha=0.01)
nb.partial_fit(X_train, y_train, classes)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [396]:
new_classes = classes.copy()
new_classes.pop()
new_classes

['B-AmountDefaulter',
 'B-CityDefaulter',
 'B-DocDateDefaulter',
 'B-EvictionDateDefaulter',
 'B-NameDefaulter',
 'B-PinCodeDefaulter',
 'B-StateDefaulter',
 'B-StreetAddDefaulter',
 'B-ThreatDefaulter',
 'I-AmountDefaulter',
 'I-DocDateDefaulter',
 'I-EvictionDateDefaulter',
 'I-NameDefaulter',
 'I-PinCodeDefaulter',
 'I-StreetAddDefaulter',
 'I-ThreatDefaulter',
 'L-AmountDefaulter',
 'L-CityDefaulter',
 'L-DocDateDefaulter',
 'L-EvictionDateDefaulter',
 'L-NameDefaulter',
 'L-PinCodeDefaulter',
 'L-StateDefaulter',
 'L-StreetAddDefaulter',
 'L-ThreatDefaulter',
 'O',
 'U-AmountDefaulter',
 'U-CityDefaulter',
 'U-DocDateDefaulter',
 'U-EvictionDateDefaulter',
 'U-NameDefaulter',
 'U-PinCodeDefaulter',
 'U-StateDefaulter',
 'U-StreetAddDefaulter']

In [397]:
test_df.head()

,doc_num,sentence_num,unique_tok_id,word
17,1,6,18,Keandris
18,1,6,19,Hogan
28,1,8,29,Georgia
29,1,8,30,30032
261,1,27,262,Served


In [398]:
test_df.drop(['doc_num', 'unique_tok_id'], axis=1, inplace=True)

In [399]:
test_transform=v.transform(test_df.to_dict('records'))

In [400]:
test_preds=nb.predict(test_transform)

In [401]:
pd.DataFrame(test_preds)[0].nunique()

15

In [402]:
from sklearn.model_selection import StratifiedKFold
import time

In [403]:
fold_n = 10
cv = fold_n   
folds = StratifiedKFold(n_splits=fold_n, random_state=10, shuffle=True)

In [404]:
y_pred_nb = {}
features = ['Sent_ID','Word']
oof = np.array(['O' for i in range(len(train_df))])

In [405]:
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())    
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]
    nb = MultinomialNB(alpha=0.01)
    nb.partial_fit(X_train, y_train, classes)
    oof[valid_index]=nb.predict(X_valid)
    y_pred_nb[fold_n]= nb.predict(test_transform)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:651: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Fold 0 started at Sun Aug 25 10:42:24 2019
Fold 1 started at Sun Aug 25 10:42:24 2019
Fold 2 started at Sun Aug 25 10:42:24 2019
Fold 3 started at Sun Aug 25 10:42:24 2019
Fold 4 started at Sun Aug 25 10:42:24 2019
Fold 5 started at Sun Aug 25 10:42:24 2019
Fold 6 started at Sun Aug 25 10:42:24 2019
Fold 7 started at Sun Aug 25 10:42:24 2019
Fold 8 started at Sun Aug 25 10:42:24 2019
Fold 9 started at Sun Aug 25 10:42:24 2019


/opt/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


In [406]:
cv_test_preds=pd.DataFrame(y_pred_nb)

In [407]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [408]:
import spacy
import en_core_web_sm
pos_tag=[]
nlp = en_core_web_sm.load()

In [409]:
from tqdm import tqdm

In [410]:
unique_words=train_df['word'].unique()

In [411]:
len(unique_words)

2561

In [412]:
pos_tag={}
for ind in tqdm(range(len(unique_words))):
    doc=nlp(unique_words[ind])
    for token in doc:
        pos_tag[unique_words[ind]]=token.tag_

100%|██████████| 2561/2561 [00:14<00:00, 174.26it/s]


In [413]:
train_words=list(pos_tag.keys())
train_pos=list(pos_tag.values())

In [414]:
pos_df=pd.DataFrame()
pos_df['word']=train_words
pos_df['pos']=train_pos

In [415]:
train_df_pos=train_df.merge(pos_df,on='word',how='left')

In [416]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['word'].values.tolist(), 
                                                           s['pos'].values.tolist(), 
                                                           s['ner'].values.tolist())]
        self.grouped = self.data.groupby('sentence_num').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(train_df_pos)
sentences = getter.sentences

In [417]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features

In [418]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [419]:
def sent2labels(sent):
    return [label for token, postag, label in sent]

In [420]:
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [421]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [422]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [423]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.05,
    c2=0.05,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X, y)
#{'c1': 0.002620608072431662, 'c2': 0.05663926338121959}

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.05, c2=0.05,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [424]:
unique_words_test=test_df['word'].unique()

In [425]:
unique_words_test=test_df['word'].unique()

In [426]:
pos_tag_test={}
for ind in tqdm(range(len(unique_words_test))):
    doc=nlp(unique_words_test[ind])
    for token in doc:
        pos_tag_test[unique_words_test[ind]]=token.tag_

100%|██████████| 430/430 [00:02<00:00, 163.95it/s]


In [427]:
test_words=list(pos_tag_test.keys())
test_pos=list(pos_tag_test.values())

In [428]:
pos_df_test=pd.DataFrame()
pos_df_test['word']=test_words
pos_df_test['pos']=test_pos

In [429]:
test_df_pos=test_df.merge(pos_df_test,on='word',how='left')

In [430]:
test_df_pos['ner']='O'

In [431]:
test_getter = SentenceGetter(test_df_pos)
test_sentences = test_getter.sentences

In [432]:
test_df_X = [sent2features(s) for s in test_sentences]
test_df_y = [sent2labels(s) for s in test_sentences]

In [433]:
y_test_pred = crf.predict(test_df_X)

In [434]:
pd.DataFrame(y_test_pred)[0].nunique()

4

In [435]:
len(y_test_pred)

94

In [436]:
y_test_pred_unpack=[val for sub_li in y_test_pred for val in sub_li]

In [437]:
len(y_test_pred_unpack)

1199

In [438]:
pred = pd.DataFrame(y_test_pred_unpack, columns={'ner'})

In [439]:
pred['ner'].nunique()

19

In [440]:
#from sklearn.metrics import f1_score

In [441]:
#f = f1_score(justfortest, pred['ner'], average='micro')

In [442]:
#f

0.8448707256046706

In [288]:
test_id=pd.DataFrame(test_id)

In [290]:
res = pd.concat([test_id, pred], axis=1)

In [293]:
res.to_csv('result3.csv', index=False)

In [57]:
# from sklearn.model_selection import RandomizedSearchCV
# import scipy
# from sklearn.metrics import make_scorer

In [137]:
# crf = sklearn_crfsuite.CRF(
#     algorithm='lbfgs',
#     #max_iterations=100,
#     all_possible_transitions=True
# )
# params_space = {
#     'c1': scipy.stats.expon(scale=0.01),
#     'c2': scipy.stats.expon(scale=0.01),
#     'max_iterations':scipy.stats.expon(scale=5)
# }

# # use the same metric for evaluation
# f1_scorer = make_scorer(metrics.flat_f1_score,
#                         average='micro')

# # search
# rs = RandomizedSearchCV(crf, params_space,
#                         cv=5,
#                         verbose=1,
#                         n_jobs=-1,
#                         n_iter=50,
#                         scoring=f1_scorer)
# rs.fit(X_train, y_train)

In [138]:
# rs.best_params_